In [2]:
from srm_helper import *
import pandas as pd
import random as rd
import numpy as np
from sklearn.model_selection import KFold 
from sklearn.metrics import r2_score
import matplotlib
matplotlib.rcParams['figure.dpi'] = 500

In [3]:
tol = .5  # MS2 fragment tolerance for QqQ optimized transitions
ppmTol = 10  # m/z tolerance for HRMS data in ppm
numCores = 2  # number of CPU cores to use
num2Train = 10 #number of compounds to learn equation
numIters = 1000 #number of iterations

In [4]:
### format csv files from whole transition list
totalTransitionInfoFn = "../data/IDX/M3T_transitions_ALTIS_optimized_allCpds.csv"

totalTransitions = pd.read_csv(totalTransitionInfoFn)

switcher = {"Positive":1,"Negative":-1}



In [5]:

allCpds = list(set(totalTransitions["Name"].values))
numCpds = len(allCpds)

In [ ]:
if __name__ == '__main__':
    
    targets = totalTransitions
    cpds = []
    toDrop = []
    for index,row in targets.iterrows():
        if (row['Name'],row["Charge"]) in cpds:
            toDrop.append(index)
        else:
            cpds.append((row['Name'],row["Charge"]))

    targets = targets.drop(toDrop)
    goodCols = ["Name","rt_start","rt_end","mz","Charge"]
    targets = targets[goodCols]
    targets.to_csv("tmp_targets_for_evaluation.csv",index=False)

    # create srm_maker object
    srm_maker = SRM_maker(ppm=ppmTol, numCores=numCores)

    msFilenames = ["../data/IDX/IDX_MS2_data/M3T_10uM_pos_DDA_10NCEs_25-35_50ms_5e4_DE5s_updatedRT.mzML",
                   "../data/IDX/IDX_MS2_data/M3T_10uM_neg_DDA_10NCEs_25-35_50ms_5e4_DE5s_updatedRT.mzML",
                   "../data/IDX/IDX_MS2_data/M3T_10uM_pos_DDA_10NCEs_25-35_80ms_1e4_DE5s_updatedRT_missing.mzML"]


    # set datafiles to build srms
    targets = pd.read_csv("tmp_targets_for_evaluation.csv")

    srm_table = pd.DataFrame()
    breakdownCurves = {}

    for msFilename in msFilenames:

        # create SRM table
        srm_table1, _ = srm_maker.createSRMsCE(msFilename, targets)

        srm_table = pd.concat((srm_table,srm_table1),axis=0,ignore_index=True)

        

Library loaded successfully: 0 spectra found
reading data...
2950  MS2 spectra detected
Number of compounds with acquired MS2:  39
Number of spectra to deconvolve:  2362


In [ ]:
transition_indices = {}
for index,row in totalTransitions.iterrows():
    new = True
    for x in transition_indices:
        if x[0] == row["Name"] and np.abs(row["Product mz"] - x[1]) < tol and row["Charge"] == x[2]:
            new = False
            transition_indices[x]["QqQ Optimized CE"] = row["CE"]
            break
    if new:
        transition_indices[(row["Name"],row["Product mz"],row["Charge"])] = {"QqQ Optimized CE":row["CE"]}
        
for index,row in srm_table.iterrows():
    new = True
    for x in transition_indices:
        if x[0] == row["Name"] and np.abs(row["Product mz"] - x[1]) < tol and row["Charge"] == x[2]:
            new = False
            transition_indices[x]["HRMS Optimized CE (converted)"] = (row["mz"], row["CE"])
            break
    if new:
        transition_indices[(row["Name"],row["Product mz"],row["Charge"])] = {"HRMS Optimized CE (converted)": (row["mz"], row["CE"])}
evaluation_results = pd.DataFrame.from_dict(transition_indices,orient="index")
evaluation_results



In [ ]:
if __name__ == "__main__":
    filt = totalTransitions
    filt.to_csv("tmp_to_learn_conv.csv",index=False)


    # create srm_maker object
    srm_maker = SRM_maker(ppm=ppmTol, numCores=numCores)

    # set datafiles for learning conversion
    trainingData = pd.read_csv("tmp_to_learn_conv.csv")

    msFilenames = ["../data/IDX/IDX_MS2_data/M3T_10uM_pos_DDA_10NCEs_25-35_50ms_5e4_DE5s_updatedRT.mzML",
                   "../data/IDX/IDX_MS2_data/M3T_10uM_neg_DDA_10NCEs_25-35_50ms_5e4_DE5s_updatedRT.mzML",
                   "../data/IDX/IDX_MS2_data/M3T_10uM_pos_DDA_10NCEs_25-35_80ms_1e4_DE5s_updatedRT_missing.mzML"]

    #build conversion
    merged = srm_maker.buildConversion(msFilenames, trainingData, tic_cutoff=0, frag_cutoff=0,
                                       frag_ppm_tolerance=2 * 1e6 * .5 / 200)
    
    merged.to_csv("../data/IDX/all_cpds_merged_to_learn_conversion.csv")
    
    print(srm_maker.getConversionEquationString())

In [ ]:
merged

In [ ]:
if __name__ == "__main__":
    #get random training compounds 
    r2s = {}
    numEval = 20
    numIters = 100
    #testCpds = rd.sample(allCpds,k=numEval)
    #trainCpds = [x for x in allCpds if x not in testCpds]
    print(len(testCpds),len(trainCpds),len(allCpds))

    for num2Train in range(2,len(trainCpds)-1):
        
        r2s[num2Train] = []
        print(num2Train)

        trainingCpds = []
        for _ in range(numIters):
            tmp = rd.sample(allCpds,k=num2Train)
            trainingCpds.append(tmp)

        for cpds in trainingCpds:
            goodInds = []
            for index,row in merged.iterrows():
                if row["Name"] in cpds:
                    goodInds.append(index)
                    
            testCpds = rd.sample([x for x in allCpds if x not in cpds],k=numEval)

            
            tmp = merged.loc[goodInds,:]
            X = np.array([tmp["mz"].values,tmp["HRMS_CE"],[1 for _ in range(len(tmp))]]).transpose()
            y = tmp["CE"].values

            #find equation
            linreg = LinearRegression(fit_intercept=False)
            linreg.fit(X,y)
            converter = lambda x: linreg.predict([[x[0],x[1],1]])[0]
            
            predCEs = []
            trueCEs = []

            for index,row in evaluation_results.iterrows():
                if index[0] in testCpds:
                    if not pd.isna(row["QqQ Optimized CE"]) and type(row["HRMS Optimized CE (converted)"]) == type(tuple()):
                        trueCEs.append(row["QqQ Optimized CE"])
                        predCEs.append(converter(row["HRMS Optimized CE (converted)"]))
                        
                        
            r2 = r2_score(trueCEs,predCEs)
            r2s[num2Train].append(r2)
            

In [ ]:
plt.hist(r2s[10])
plt.xlabel("R2")
plt.ylabel("frequency")
plt.savefig("r2_hist_at_10cpds.png")

In [ ]:
print(r2s.keys())

In [ ]:
keys = list(r2s.keys())
keys.sort()
keys = keys[4:]
vals = [np.mean(r2s[k]) for k in keys]
errs = [np.std(r2s[k]) for k in keys]
plt.scatter(keys,vals,color="black")
plt.errorbar(keys,vals,yerr=errs,color="black",capsize=3)
plt.plot(keys,vals)
#plt.ylim((0,1.2))
plt.xlabel("# of compounds")
plt.ylabel("R2")
#r2 = r2_score(merged["CE"],merged["HRMS_CE"])
#plt.plot(keys,[r2 for _ in keys],color="red",label="no conversion")
plt.savefig("num_cpds_plot.png")